### ISR Naive Approach 
- use assistant api, upload the entire document and do QA on entire document
- https://platform.openai.com/docs/assistants/how-it-works/objects
- https://platform.openai.com/docs/api-reference/assistants

In [1]:
import os,sys,getpass
sys.path.insert(0,'../../libs')
import time
from utils import load_json,logging,exception_handler
from llm_utils import get_oai_fees
import json
import openai
from openai import OpenAI
## load API Key
#key = load_json('/root/workspace/Dev/openai_key.json') 
#os.environ['OPENAI_API_KEY'] = key['ChatGPT1']['API_KEY']
os.environ["OPENAI_API_KEY"] = getpass.getpass(prompt='OpenAI API Token:')


In [2]:
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

In [7]:
## upload a file 
def upload_file(client,up_file_path):
    assert os.path.exists(up_file_path)
    file = client.files.create(
        file=open(up_file_path,'rb'),
        purpose='assistants'
    )
    return(file.id)

In [8]:
up_file_path = '/root/workspace/data/DOCs/PDF/USA_2022.pdf'
usa_file_id = upload_file(client,up_file_path)
print(usa_file_id)

In [12]:
print(client.files.list())

SyncPage[FileObject](data=[FileObject(id='file-bPuZpeGbCK6bN24DVyAE4Epd', bytes=3466567, created_at=1704250068, filename='USA_2022.pdf', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-puUNnu0XOnn8wazwphqSb2Jh', bytes=70886, created_at=1703462607, filename='paper.mmd', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-5tTWpLqP9KDOvf0wSdUZLt48', bytes=70886, created_at=1703462320, filename='paper.mmd', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-Ge0W1Zby8GliSvzlHd3I62rZ', bytes=70886, created_at=1703461727, filename='paper.mmd', object='file', purpose='assistants', status='processed', status_details=None), FileObject(id='file-mYHLNzsxghkkFr2lMNgEFfUo', bytes=300597, created_at=1702931200, filename='Consolidated BRICS Summit Statements_2009-2023.docx', object='file', purpose='assistants', status='processed', status_details=None), FileO

In [13]:
# Add the file to the assistant 
assistant_usa = client.beta.assistants.create(
    instructions="You are an experienced IMF economist. Your main job is to review and analyze IMF country staff reports. Use your knowledge base to best respond to user questions",
    model="gpt-4-1106-preview",
    tools=[{"type":"retrieval"}],
    file_ids = [usa_file_id]
)
## you can use client.beta.assistants.update if you need to change some of the settings
print(assistant_usa.id)

asst_sOY7glbeM2iIq8n7EWKlewV7


In [14]:
# create a thread 
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_N7fCWBnVZGIDeW3cPlnJjcXy', created_at=1704250418, metadata={}, object='thread')


In [15]:
prompt_identify_risks = """
You are an economist in the IMF. Your main task is to review Country Staff reports. 
Please read carefully the document. Please determine all major macro critical downside or upside risks described in the document. Pay more attention to common macro critical risks like inflation, geopolitics, trade, climate etc. 
Please return all identified risks in a JSON object. 

Output is a JSON object with the following format:
"risks": [
    {{"risk_name": "<risk1>", "risk_description": "<risk_description1>"}}, 
    {{"risk_name": "<risk2>", "risk_description": "<risk_description2>"}},
    ......
    ]

Please proceed with the task, keeping in mind the importance of accuracy and clarity in your analysis. Return the JSON object only and nothing else. 

"""

In [16]:
## create message and add to the thread 
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt_identify_risks
)

In [17]:
## run the assistant 
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant_usa.id,
    #instructions="You are an economist in the IMF. Your main task is to review Country Staff reports."
)

print(run.id)

run_86psMX1AmxbnlZzzfhlWP5Sz


In [18]:
def wait_for_run_completion(thread_id, run_id):
    while True:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run_id)
        print(f"Current run status: {run.status}")
        if run.status in ['completed', 'failed', 'requires_action']:
            return run

run = wait_for_run_completion(thread.id,run.id)

Current run status: completed


In [20]:
## retrieve the message 
return_messages = client.beta.threads.messages.list(
  thread_id=thread.id
)
# for each in return_messages:
#   print(each.role+": {}".format(each.content[0].text.value))
#   print("=============")
print(return_messages.data[0].content[0].text.value)

```json
{
  "risks": [
    {"risk_name": "Geopolitical tension and sanctions", "risk_description": "Russia's invasion of Ukraine leading to escalation of sanctions and other disruptions, which result in higher commodity prices, refugee migration, tighter financial conditions, and other adverse spillovers, particularly affecting LICs and commodity-importing EMs【11†source】."},
    {"risk_name": "Commodity price volatility", "risk_description": "Rising and volatile food and energy prices due to supply constraints, war in Ukraine, export restrictions, and currency depreciations, causing disruptions in the green transition, price and real sector volatility, food insecurity, social unrest, and acute crises, especially in EMDEs with lack of fiscal space【12†source】."},
    {"risk_name": "China's economic slowdown", "risk_description": "Abrupt growth slowdown in China due to extended COVID-19 lockdowns, rising geopolitical tensions, property sector slowdown, or inadequate policy responses, with